In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pyswarms as ps
import joblib

In [2]:
print("Memulai proses optimasi K-NN untuk model MULTI-KELAS...")

# 1. Muat Dataset dari file Excel multi-kelas
try:
    df = pd.read_excel('data_daun_multiclass.xlsx')
except FileNotFoundError:
    print("Error: File 'data_daun_multiclass.xlsx' tidak ditemukan.")
    print("Pastikan Anda sudah menjalankan skrip '1_ekstraksi_fitur_multiclass.py' terlebih dahulu.")
    exit()

# Pisahkan fitur (X) dan label (y)
X = df[['rata_R', 'rata_G', 'rata_B']]
y = df['label']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\nData berhasil dibagi:")
print(f"- Jumlah Data Latih: {len(X_train)} baris")
print(f"- Jumlah Data Uji: {len(X_test)} baris")
print(f"- Jumlah kelas: {len(y.unique())}")

# Menyiapkan file Excel untuk menyimpan data latih dan uji
output_excel_file = 'hasil_optimasi_multiclass.xlsx'
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)
with pd.ExcelWriter(output_excel_file) as writer:
    df_train.to_excel(writer, sheet_name='Data Latih', index=False)
    df_test.to_excel(writer, sheet_name='Data Uji', index=False)

print(f"\nData Latih dan Uji awal telah disimpan ke '{output_excel_file}'")

# 2. Buat Fungsi Fitness untuk PSO
def fitness_function(params):
    k = int(params[0])
    if k % 2 == 0: k += 1
    
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    error = 1.0 - accuracy
    return error

# 3. Konfigurasi dan Jalankan PSO
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
bounds = (np.array([1]), np.array([31]))
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=1, options=options, bounds=bounds)
cost, pos = optimizer.optimize(fitness_function, iters=20)

# 4. Ambil Hasil Terbaik
k_terbaik = int(pos[0])
if k_terbaik % 2 == 0: k_terbaik += 1
akurasi_terbaik = 1.0 - cost

print("\nOptimasi Selesai!")
print(f"Nilai K terbaik yang ditemukan oleh PSO: {k_terbaik}")
print(f"Akurasi yang dicapai pada data uji: {akurasi_terbaik:.4f}")

# 5. Latih model dengan K terbaik dan simpan hasil prediksi
final_knn_model = KNeighborsClassifier(n_neighbors=k_terbaik)
final_knn_model.fit(X_train, y_train)
y_hasil_prediksi = final_knn_model.predict(X_test)

# Tambahkan kolom 'Prediksi_Model' ke DataFrame data uji
df_test_hasil = df_test.copy()
df_test_hasil['Prediksi_Model'] = y_hasil_prediksi

print("\nMenambahkan hasil prediksi ke file Excel...")
with pd.ExcelWriter(output_excel_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_test_hasil.to_excel(writer, sheet_name='Data Uji', index=False)

# 6. Latih model final dengan SEMUA data dan simpan
print("Melatih model final dengan semua data...")
final_model_all_data = KNeighborsClassifier(n_neighbors=k_terbaik)
final_model_all_data.fit(X, y)
joblib.dump(final_model_all_data, 'model_multiclass.pkl')


print("\nProses Selesai!")
print(f"File '{output_excel_file}' telah diperbarui dengan hasil prediksi.")
print("Model final 'model_multiclass.pkl' juga telah dibuat dan siap digunakan.")

Memulai proses optimasi K-NN untuk model MULTI-KELAS...

Data berhasil dibagi:
- Jumlah Data Latih: 1310 baris
- Jumlah Data Uji: 562 baris
- Jumlah kelas: 6


2025-09-17 01:48:17,533 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}



Data Latih dan Uji awal telah disimpan ke 'hasil_optimasi_multiclass.xlsx'


pyswarms.single.global_best:   0%|          |0/20C:\Users\TEGAR\AppData\Local\Temp\ipykernel_13208\3201346595.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  k = int(params[0])
pyswarms.single.global_best:   0%|          |0/20, best_cost=0.221C:\Users\TEGAR\AppData\Local\Temp\ipykernel_13208\3201346595.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  k = int(params[0])
pyswarms.single.global_best:   0%|          |0/20, best_cost=0.221C:\Users\TEGAR\AppData\Local\Temp\ipykernel_13208\3201346595.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a 


Optimasi Selesai!
Nilai K terbaik yang ditemukan oleh PSO: 31
Akurasi yang dicapai pada data uji: 0.7794

Menambahkan hasil prediksi ke file Excel...
Melatih model final dengan semua data...

Proses Selesai!
File 'hasil_optimasi_multiclass.xlsx' telah diperbarui dengan hasil prediksi.
Model final 'model_multiclass.pkl' juga telah dibuat dan siap digunakan.
